<a href="https://colab.research.google.com/github/VishyTheFishy/ddim/blob/main/CTtoMRI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install diffusers
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.6 MB/s eta 0:00:00
  Created wheel for diffusers: filename=diffusers-0.21.2-py3-none-any.whl size=1489250 sha256=59b9d01d8104cae83c605cb4f06a47d9584a120a154920577d2151414ab6b70a
  Stored in directory: /root/.cache/pip/wheels/2e/09/32/11c9e42c397d3f3494226b28ba68c4ad4718a68a65dba14ea6
Successfully built diffusers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.3 MB/s eta 0:00:00


In [12]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image

# Step 3: Create a custom dataset class
class CIFAR10WithGray(Dataset):
    def __init__(self, root, train=True, transform=None):
        self.cifar_dataset = torchvision.datasets.CIFAR10(root=root, train=train, download=True, transform=transform)

    def __len__(self):
        return len(self.cifar_dataset)

    def __getitem__(self, idx):
        color_image, label = self.cifar_dataset[idx]

        # Convert the color image to grayscale
        grayscale_image = self.rgb_to_grayscale(color_image)


        return color_image, grayscale_image

    def rgb_to_grayscale(self, img):
        # Convert a color (RGB) PIL image to grayscale
        gray = torchvision.transforms.Grayscale()(img)
        return torch.cat((gray,gray,gray))

# Step 2: Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a common size
    transforms.ToTensor(),         # Convert to tensor
])

# Step 4: Instantiate DataLoader
train_dataset = CIFAR10WithGray(root='./data', train=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=4)


Files already downloaded and verified


In [2]:
from transformers import  ViTModel
import torch


MRIEncoder = ViTModel.from_pretrained("google/vit-base-patch16-224").to("cuda")
CTEncoder = ViTModel.from_pretrained("google/vit-base-patch16-224").to("cuda")

MRIOptim = torch.optim.Adam(MRIEncoder.parameters())
CTOptim = torch.optim.Adam(CTEncoder.parameters())

for i, data in enumerate(train_loader):
    MRIOptim.zero_grad()
    CTOptim.zero_grad()
    ct, mri = data
    print(ct.shape,mri.shape)
    ctEncoding = CTEncoder(ct.to("cuda")).last_hidden_state
    mriEncoding = MRIEncoder(mri.to("cuda")).last_hidden_state
    loss_fn = torch.nn.MSELoss()
    loss = loss_fn(ctEncoding,mriEncoding)
    loss.backward()
    MRIOptim.step()
    CTOptim.step()





Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NameError: ignored

First we build an encoding using paired data. Then we train a diffusion model to convert the encoding to an MRI using all unpaired MRI. Finally convert encodings from CT into MRI.

---

